In [3]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv',error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

glancing at the data:

In [4]:
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [18]:
import gensim 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to /home/grace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


A function to perform lemmatize and stem preprocessing steps on the data set:

In [20]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


picking a doc to preview after preprocessing:

In [21]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


Preprocess the headline text, saving the results as ‘processed_docs’

In [23]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [25]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


"Filter out everything that's dumb"
- tokens that appear in less than 15 docs
- tokens that appear in more half of the docs
- keep only the first 100000 most frequent tokens


In [27]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [28]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

preview our bag o' words:

In [29]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.
tf-idf stands for "term frequency inverse document frequency". It is an idicator of the importance of a word. A high score means the word is important, and a low score means less important. It's based on the idea that a word that is very frequent in an individual doc and is very frequent over the entire corpus of documents is unimportant (ex. the, of). 

In [31]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [32]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [33]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"bushfir" + 0.026*"hous" + 0.021*"south" + 0.019*"north" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.011*"gippsland" + 0.011*"east" + 0.010*"coast"
Topic: 1 
Words: 0.030*"kill" + 0.028*"shoot" + 0.025*"dead" + 0.020*"polic" + 0.019*"protest" + 0.019*"attack" + 0.013*"offic" + 0.013*"assault" + 0.012*"chines" + 0.011*"michael"
Topic: 2 
Words: 0.056*"australia" + 0.045*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.010*"return" + 0.010*"beat"
Topic: 3 
Words: 0.032*"polic" + 0.027*"charg" + 0.025*"court" + 0.023*"death" + 0.023*"murder" + 0.023*"woman" + 0.022*"victoria" + 0.016*"victorian" + 0.016*"face" + 0.016*"alleg"
Topic: 4 
Words: 0.020*"resid" + 0.019*"emerg" + 0.018*"chang" + 0.018*"say" + 0.015*"fear" + 0.014*"speak" + 0.014*"power" + 0.013*"worker" + 0.011*"concern" + 0.011*"climat"
Topic: 5 
Words: 0.027*"news" + 0.021*"live" + 0.021*"market" + 0.020*"tasmania" + 0.018*"women" 

running LDA using TF-IDF:

In [34]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.028*"trump" + 0.009*"leagu" + 0.009*"world" + 0.008*"final" + 0.007*"rugbi" + 0.006*"open" + 0.006*"australia" + 0.006*"beat" + 0.005*"fiji" + 0.005*"game"
Topic: 1 Word: 0.017*"crash" + 0.012*"kill" + 0.010*"dead" + 0.010*"die" + 0.007*"rescu" + 0.007*"north" + 0.007*"korea" + 0.007*"injur" + 0.006*"polic" + 0.006*"truck"
Topic: 2 Word: 0.016*"murder" + 0.016*"charg" + 0.014*"polic" + 0.014*"court" + 0.011*"alleg" + 0.010*"death" + 0.009*"royal" + 0.009*"woman" + 0.009*"jail" + 0.009*"sentenc"
Topic: 3 Word: 0.011*"govern" + 0.008*"price" + 0.007*"market" + 0.007*"australian" + 0.007*"rise" + 0.006*"share" + 0.006*"dollar" + 0.005*"live" + 0.005*"cattl" + 0.005*"violenc"
Topic: 4 Word: 0.013*"elect" + 0.007*"labor" + 0.007*"monday" + 0.007*"thursday" + 0.006*"wall" + 0.006*"sport" + 0.006*"feder" + 0.006*"peter" + 0.006*"green" + 0.006*"say"
Topic: 5 Word: 0.014*"drum" + 0.009*"tuesday" + 0.009*"michael" + 0.007*"juli" + 0.006*"august" + 0.006*"cancer" + 0.006*"explai

Performance evaluation by classifying sample document using LDA Bag of Words model

We will check where our test document would be classified.

In [35]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [36]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4559560716152191	 
Topic: 0.019*"donald" + 0.015*"beach" + 0.014*"nation" + 0.014*"farmer" + 0.013*"rural" + 0.013*"time" + 0.012*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan"

Score: 0.444010466337204	 
Topic: 0.034*"govern" + 0.025*"stori" + 0.019*"warn" + 0.018*"feder" + 0.016*"health" + 0.014*"countri" + 0.014*"fund" + 0.014*"claim" + 0.013*"life" + 0.012*"say"

Score: 0.012506913393735886	 
Topic: 0.034*"elect" + 0.017*"water" + 0.017*"state" + 0.015*"tasmanian" + 0.011*"labor" + 0.011*"liber" + 0.010*"morrison" + 0.010*"leader" + 0.010*"parti" + 0.009*"campaign"

Score: 0.012504363432526588	 
Topic: 0.020*"resid" + 0.019*"emerg" + 0.018*"chang" + 0.018*"say" + 0.015*"fear" + 0.014*"speak" + 0.014*"power" + 0.013*"worker" + 0.011*"concern" + 0.011*"climat"

Score: 0.012503700330853462	 
Topic: 0.028*"bushfir" + 0.026*"hous" + 0.021*"south" + 0.019*"north" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.011*"gippsland" + 0.011*"east" + 0.010*"coast"

S

try one more:

In [43]:
processed_docs[17]


['british', 'combat', 'troop', 'arriv', 'daili', 'kuwait']

In [44]:
for index, score in sorted(lda_model[bow_corpus[17]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3002224862575531	 
Topic: 0.030*"kill" + 0.028*"shoot" + 0.025*"dead" + 0.020*"polic" + 0.019*"protest" + 0.019*"attack" + 0.013*"offic" + 0.013*"assault" + 0.012*"chines" + 0.011*"michael"

Score: 0.2996271252632141	 
Topic: 0.028*"bushfir" + 0.026*"hous" + 0.021*"south" + 0.019*"north" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.011*"gippsland" + 0.011*"east" + 0.010*"coast"

Score: 0.15723927319049835	 
Topic: 0.049*"year" + 0.043*"trump" + 0.043*"sydney" + 0.027*"queensland" + 0.024*"peopl" + 0.021*"home" + 0.020*"adelaid" + 0.018*"perth" + 0.015*"brisban" + 0.015*"royal"

Score: 0.15711906552314758	 
Topic: 0.019*"donald" + 0.015*"beach" + 0.014*"nation" + 0.014*"farmer" + 0.013*"rural" + 0.013*"time" + 0.012*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan"

Score: 0.014298676513135433	 
Topic: 0.056*"australia" + 0.045*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.010*"return" + 0.010

Performance evaluation by classifying sample document using LDA TF-IDF model:
- we like tf-idf because we think it is classifiying better

In [46]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.41654595732688904	 
Topic: 0.011*"govern" + 0.008*"price" + 0.007*"market" + 0.007*"australian" + 0.007*"rise" + 0.006*"share" + 0.006*"dollar" + 0.005*"live" + 0.005*"cattl" + 0.005*"violenc"

Score: 0.3374316394329071	 
Topic: 0.013*"elect" + 0.007*"labor" + 0.007*"monday" + 0.007*"thursday" + 0.006*"wall" + 0.006*"sport" + 0.006*"feder" + 0.006*"peter" + 0.006*"green" + 0.006*"say"

Score: 0.15848049521446228	 
Topic: 0.024*"news" + 0.020*"countri" + 0.020*"rural" + 0.015*"hour" + 0.010*"search" + 0.009*"friday" + 0.009*"nation" + 0.009*"scott" + 0.009*"turnbul" + 0.007*"miss"

Score: 0.012506398372352123	 
Topic: 0.010*"wednesday" + 0.009*"care" + 0.009*"morrison" + 0.008*"hill" + 0.008*"financ" + 0.008*"energi" + 0.007*"octob" + 0.007*"age" + 0.007*"june" + 0.006*"disabl"

Score: 0.012506279163062572	 
Topic: 0.014*"drum" + 0.009*"tuesday" + 0.009*"michael" + 0.007*"juli" + 0.006*"august" + 0.006*"cancer" + 0.006*"explain" + 0.005*"footag" + 0.005*"america" + 0.005*"kohl

Real deal: testing on unseen document

In [51]:
unseen_document = 'Bushfire kills cattle herd in queensland'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5352160930633545	 Topic: 0.032*"polic" + 0.027*"charg" + 0.025*"court" + 0.023*"death" + 0.023*"murder"
Score: 0.3314177989959717	 Topic: 0.030*"kill" + 0.028*"shoot" + 0.025*"dead" + 0.020*"polic" + 0.019*"protest"
Score: 0.01667379029095173	 Topic: 0.034*"govern" + 0.025*"stori" + 0.019*"warn" + 0.018*"feder" + 0.016*"health"
Score: 0.016670752316713333	 Topic: 0.034*"elect" + 0.017*"water" + 0.017*"state" + 0.015*"tasmanian" + 0.011*"labor"
Score: 0.01667063497006893	 Topic: 0.056*"australia" + 0.045*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test"
Score: 0.016670599579811096	 Topic: 0.020*"resid" + 0.019*"emerg" + 0.018*"chang" + 0.018*"say" + 0.015*"fear"
Score: 0.01667056605219841	 Topic: 0.049*"year" + 0.043*"trump" + 0.043*"sydney" + 0.027*"queensland" + 0.024*"peopl"
Score: 0.016669999808073044	 Topic: 0.027*"news" + 0.021*"live" + 0.021*"market" + 0.020*"tasmania" + 0.018*"women"
Score: 0.016669999808073044	 Topic: 0.019*"donald" + 0.015*"beach" + 0.014

### Conclusion
We think this didn't work as well as we wanted it to because we trained it on a corpus of data that covered a wide range of topics. If we had a corpus on one topic, like JUST political tweets, the model might work better.